In [1]:
#BFS DFS

#include <iostream>
#include <omp.h>
#include <vector>
#include <stack>

using namespace std;

struct Node{
    int id;
    vector<int> neighbors;
    bool visited;
};

void parallelDfs(vector<Node> &graph, int source){
    int numNodes=graph.size();
    
    stack<int> dfsStack;
    
    graph[source].visited = true;
    dfsStack.push(source);
    
    #pragma omp parallel
    {
        
        while(!dfsStack.empty()){
        int current = dfsStack.top();
        dfsStack.pop();
        #pragma omp for
        for(int i=0;i<graph[current].neighbors.size();++i){
            int neighborID = graph[current].neighbors[i];
            if(!graph[neighborID].visited)
            #pragma omp critical
            {
                graph[neighborID].visited = true;
                dfsStack.push(neighborID);
            }
        }
        }
        
    }
}

void addUndirectedEdge(vector<Node>& graph, int src, int dest){
    graph[src].neighbors.push_back(dest);
    graph[dest].neighbors.push_back(src);
}

int main() {
    vector<Node> graph(8);
    addUndirectedEdge(graph,0,1);
    addUndirectedEdge(graph,0,2);
    addUndirectedEdge(graph,1,3);
    addUndirectedEdge(graph,1,4);
    addUndirectedEdge(graph,2,5);
    addUndirectedEdge(graph,2,6);
    addUndirectedEdge(graph,4,7);
    
    parallelDfs(graph,0);
    cout<< "Visited Nodes: ";
    for(int i = 0;i < graph.size(); ++i){
        if(graph[i].visited){
            cout<<i<<" ";
        }
    }
    return 0;
}

SyntaxError: invalid syntax (3561099985.py, line 8)

In [1]:
# parallelBUBBLE

#include<iostream>
#include<vector>
#include<omp.h>

using namespace std;

void paralledBubbleSort(vector<int> &arr){
    int n=arr.size();
    bool swapped;
    for(int i =0 ;i < n;++i){
        swapped=false;
        #pragma omp parallel for shared(arr,swapped)
        for(int j = 0;j<n-i-1;++j){
            if(arr[j]>arr[j+1]){
                swap(arr[j],arr[j+1]);
                swapped = true;
            }
        }
        if(!swapped){
            break;
        }
    }
}

int main(){
    vector<int> arr = {2,3,4,5,12,1,2,3};
    
    paralledBubbleSort(arr);
    cout<<"Sorted Arr: ";
    for(int i=0;i<arr.size();++i){
        cout<<arr[i]<<" ";
    }
}


SyntaxError: invalid syntax (433158055.py, line 7)

In [2]:
#Merge Sort
#include<vector>
#include<iostream>
#include<omp.h>

using namespace std;

void merge(vector<int> &arr,int left,int mid, int right){
    int n1 = mid - left + 1;
    int n2 = right - mid ;
    
    vector<int> L(n1), R(n2);
    
    for(int i=0;i<n1;i++){
        L[i] = arr[left + i];
    }
    
    for(int j=0;j<n2;++j){
        R[j] = arr[mid + j + 1];
    }
    
    int i = 0, j = 0, k=left;
    
    while(i<n1 && j<n2){
        if(L[i]<=R[j]){
            arr[k] = L[i];
            ++i;
        }
        else{
            arr[k] = R[j];
            ++j;
        }
        ++k;
    }
    
    while(i<n1){
        arr[k] = L[i];
        ++i;
        ++k;
    }
    
    while(j<n2){
        arr[k] = R[j];
        ++j;
        ++k;
    }
    
}

void parallelMergeSort(vector<int> &arr, int left, int right){
    if(left<right){
        int mid = left + (right - left)/2;
        
        #pragma omp parallel sections
        {
            
            #pragma omp section
            {
                parallelMergeSort(arr,left,mid);
            }
            
            #pragma omp section
            {
                parallelMergeSort(arr,mid+1,right);
            }
        }
        merge(arr,left,mid,right);
    }
}


int main(){
    
    vector<int> arr = {1,2,4,5,34,14,2,3,34,5,3,6,76,56};
    
    parallelMergeSort(arr,0,13);
    
    cout<<"SORTED: ";
    
    for(int i = 0 ;i < arr.size(); i++){
        cout<<arr[i]<<" ";
    }
}

SyntaxError: invalid syntax (3611801081.py, line 6)

In [ ]:
#MIN MAX
#include<vector>
#include<iostream>
#include<omp.h>

using namespace std;

int minArr(vector<int> &arr){
    int minVal = arr[0];
    
    #pragma omp parallel reduction(min: minVal)
    for(int i = 0;i < arr.size();i++){
        if(arr[i]<minVal){
            minVal = arr[i];
        }
    }
    
    return minVal;
    
}

int maxArr(vector<int> &arr){
    int maxVal = arr[0];
    
    #pragma omp parallel reduction(max: maxVal)
    for(int i = 0;i < arr.size();i++){
        if(arr[i]>maxVal){
            maxVal = arr[i];
        }
    }
    
    return maxVal;
    
}

int sumArr(vector<int> &arr){
    int sum = 0;
    
    #pragma omp parallel reduction(+: sum)
    for(int i = 0;i < arr.size();i++){
        sum = sum+arr[i];
    }
    
    return sum;
    
}

int avgArr(vector<int> &arr){
    int sum = sumArr(arr);
    int avg = sum/arr.size();
    
    return avg;
    
}


int main(){
    
    vector<int> arr = {1,2,4,5,34,14,2,3,34,5,3,6,76,56};
    
    cout<<"MIN: "<<minArr(arr)<<"\n";
    cout<<"MAX: "<<maxArr(arr)<<"\n";
    cout<<"SUM: "<<sumArr(arr)<<"\n";
    cout<<"AVG: "<<avgArr(arr)<<"\n";
    
}